In [1]:
from pyetp.client import connect
from pyetp.config import SETTINGS
from pyetp.types import *
import xtgeo

SETTINGS.dataspace = 'demo/pss-data-gateway'
SETTINGS.etp_url = 'wss://<URL>'
SETTINGS.application_name ='etpTest'
SETTINGS.application_version='0.0.1'

token = ""
async with connect(authorization=token, timeout=60) as client:
        await client.put_dataspaces_no_raise(client.default_dataspace_uri) # create new dataspace if not exist
        surf = xtgeo.surface_from_file('../data/test.gri', fformat='irap_binary')
        epsg_code = 23031
        urls = await client.put_xtgeo_surface(surf, epsg_code, client.default_dataspace_uri) # chunked upload using subarray if size too large


DEBUG:pyetp.client:sending RequestSession MessageHeader(protocol=0, message_type=1, correlation_id=0, message_id=2, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:starting recv loop
DEBUG:pyetp.client:recv OpenSession MessageHeader(protocol=0, message_type=2, correlation_id=2, message_id=1, message_flags=2)
DEBUG:pyetp.client:sending PutDataspaces MessageHeader(protocol=24, message_type=3, correlation_id=0, message_id=4, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv ProtocolException MessageHeader(protocol=24, message_type=1000, correlation_id=4, message_id=3, message_flags=2)
DEBUG:pyetp.client:sending PutDataObjects MessageHeader(protocol=4, message_type=2, correlation_id=0, message_id=6, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv PutDataObjectsResponse MessageHeader(protocol=4, message_type=9, correlation_id=6, message_id=5, message_flags=2)
DEBUG:pyetp.client:sending PutDataArrays MessageHeader(protocol=9, message_type=4, 

In [2]:
for i in urls:
    print(i.raw_uri)


eml:///dataspace('demo/pss-data-gateway')/eml20.EpcExternalPartReference(13a747a5-4350-4ab1-92cb-05d587e3b909)
eml:///dataspace('demo/pss-data-gateway')/resqml20.LocalDepth3dCrs(211a7405-c702-4a15-946e-c183d80581f3)
eml:///dataspace('demo/pss-data-gateway')/resqml20.Grid2dRepresentation(d2282613-d5f7-45fb-a07a-bfefdec18013)


In [3]:
async with connect(authorization=token, timeout=60) as client:
    gri, = await client.get_resqml_objects(urls[2]) # parallelized using subarray
    arr = await client.get_array(DataArrayIdentifier(
                uri=str(urls[0]), pathInResource=gri.grid2d_patch.geometry.points.zvalues.values.path_in_hdf_file
            )) # parallelized using subarray

DEBUG:pyetp.client:sending RequestSession MessageHeader(protocol=0, message_type=1, correlation_id=0, message_id=2, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:starting recv loop
DEBUG:pyetp.client:recv OpenSession MessageHeader(protocol=0, message_type=2, correlation_id=2, message_id=1, message_flags=2)
DEBUG:pyetp.client:sending GetDataObjects MessageHeader(protocol=4, message_type=1, correlation_id=0, message_id=4, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv GetDataObjectsResponse MessageHeader(protocol=4, message_type=4, correlation_id=4, message_id=3, message_flags=2)
DEBUG:pyetp.client:sending GetDataArrayMetadata MessageHeader(protocol=9, message_type=6, correlation_id=0, message_id=6, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv GetDataArrayMetadataResponse MessageHeader(protocol=9, message_type=7, correlation_id=6, message_id=5, message_flags=2)
DEBUG:pyetp.client:sending GetDataArrays MessageHeader(protocol=9, mes

In [4]:
gri

Grid2dRepresentation(citation=Citation(title='unknown', originator='etpTest', creation=XmlDateTime(2024, 3, 21, 9, 59, 48), format='etpTest:v0.0.1', editor=None, last_update=None, version_string=None, description=None, descriptive_keywords=None), aliases=[], custom_data=None, schema_version='2.0', uuid='d2282613-d5f7-45fb-a07a-bfefdec18013', object_version=None, extra_metadata=[], represented_interpretation=None, surface_role=<SurfaceRole.MAP: 'map'>, boundaries=[], grid2d_patch=Grid2dPatch(patch_index=0, fastest_axis_count=400, slowest_axis_count=500, geometry=PointGeometry(time_index=None, local_crs=DataObjectReference(content_type='application/x-resqml+xml;version=2.0;type=LocalDepth3dCrs', title='CRS for unknown', uuid='211a7405-c702-4a15-946e-c183d80581f3', uuid_authority=None, version_string=None), points=Point3dZValueArray(supporting_geometry=Point3dLatticeArray(all_dimensions_are_orthogonal=None, origin=Point3d(coordinate1=414.38568115234375, coordinate2=679.4137573242188, coor

In [5]:
arr

array([[0.8506015 , 0.56227505, 0.31100875, ..., 0.82323563, 0.44379264,
        0.7478812 ],
       [0.75474274, 0.02914623, 0.48869565, ..., 0.224014  , 0.6695323 ,
        0.86974293],
       [0.38983092, 0.92269933, 0.9598281 , ..., 0.20948039, 0.22134982,
        0.00741855],
       ...,
       [0.72381335, 0.31973526, 0.7490787 , ..., 0.913171  , 0.05440214,
        0.9691767 ],
       [0.5557336 , 0.10602916, 0.46710327, ..., 0.2674649 , 0.8692388 ,
        0.07431769],
       [0.5399276 , 0.5520293 , 0.43321192, ..., 0.25207523, 0.55928504,
        0.6851172 ]], dtype=float32)

In [6]:
async with connect(authorization=token, timeout=60) as client:
    surf = await client.get_xtgeo_surface(urls[0], urls[2])

DEBUG:pyetp.client:sending RequestSession MessageHeader(protocol=0, message_type=1, correlation_id=0, message_id=2, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:starting recv loop
DEBUG:pyetp.client:recv OpenSession MessageHeader(protocol=0, message_type=2, correlation_id=2, message_id=1, message_flags=2)
DEBUG:pyetp.client:sending GetDataObjects MessageHeader(protocol=4, message_type=1, correlation_id=0, message_id=4, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv GetDataObjectsResponse MessageHeader(protocol=4, message_type=4, correlation_id=4, message_id=3, message_flags=2)
DEBUG:pyetp.client:sending GetDataArrayMetadata MessageHeader(protocol=9, message_type=6, correlation_id=0, message_id=6, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:recv GetDataArrayMetadataResponse MessageHeader(protocol=9, message_type=7, correlation_id=6, message_id=5, message_flags=2)
DEBUG:pyetp.client:sending GetDataArrays MessageHeader(protocol=9, mes

In [7]:
async with connect(authorization=token, timeout=60) as client:
    urls = await client.delete_data_objects(*map(str, urls))

DEBUG:pyetp.client:sending RequestSession MessageHeader(protocol=0, message_type=1, correlation_id=0, message_id=2, message_flags=<MessageFlags.FINALPART: 2>)
DEBUG:pyetp.client:starting recv loop
DEBUG:pyetp.client:recv OpenSession MessageHeader(protocol=0, message_type=2, correlation_id=2, message_id=1, message_flags=2)
DEBUG:pyetp.client:sending DeleteDataObjects MessageHeader(protocol=4, message_type=3, correlation_id=0, message_id=4, message_flags=<MessageFlags.FINALPART: 2>)


DEBUG:pyetp.client:recv DeleteDataObjectsResponse MessageHeader(protocol=4, message_type=10, correlation_id=4, message_id=3, message_flags=2)
DEBUG:pyetp.client:sending CloseSession MessageHeader(protocol=0, message_type=5, correlation_id=0, message_id=6, message_flags=<MessageFlags.FINALPART: 2>)
